### API Portion of Code

In [4]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials
import sys
import json

from config import client_id
from config import client_secret
from config import redirect_uri

In [4]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id,
                                               client_secret,
                                               redirect_uri,
                                               scope="user-library-read"))

results = sp.current_user_saved_tracks()
for idx, item in enumerate(results['items']):
    track = item['track']
    print(idx, track['artists'][0]['name'], " – ", track['name'])

0 Bausa  –  Marlboro Lights
1 FOURTY  –  Heimkommen
2 CRO  –  oneway
3 Bausa  –  Baron
4 Bausa  –  Tropfen
5 Gary Washington  –  Wolkenkratzer
6 The Cratez  –  Skifahren
7 CRO  –  ALLES DOPE
8 CRO  –  HÖR NICHT AUF (feat. Teesy)
9 Rammstein  –  Dicke Titten
10 Zedline  –  Polozhenie - Zedline Remix
11 Bad Religion  –  A Walk
12 Peyton Parrish  –  My Mother Told Me - Old Norse Version
13 Peyton Parrish  –  Shadow of You
14 TOOL  –  Opiate²
15 SKÁLD  –  Þat Mælti Mín Móðir
16 Hindarfjäll  –  Þat Mælti Mín Móðir
17 LocSta  –  Положение (Dior)
18 The Weeknd  –  Sacrifice
19 Marteria  –  Paradise Delay


In [ ]:
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
from pprint import pprint

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id,
                                               client_secret,
                                               redirect_uri,))

pl_id = 'spotify:playlist:37i9dQZEVXbMDoHDwVN2tF'
offset = 0

while True:
    response = sp.playlist_items(pl_id,
                                 offset=offset,
                                 fields='items.track.id,items.track.name,items.track.album.name,items.track.plays,total',
                                 additional_types=['track'])
    
    if len(response['items']) == 0:
        break
    
    pprint(response['items'])
    offset = offset + len(response['items'])
    print(offset, "/", response['total'])

In [ ]:
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
from pprint import pprint

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id,
                                               client_secret,
                                               redirect_uri,))

pl_id = 'spotify:playlist:37i9dQZEVXbMDoHDwVN2tF'

results = sp.playlist(pl_id)
print(json.dumps(results, indent=4))

In [ ]:
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
from pprint import pprint

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id,
                                               client_secret,
                                               redirect_uri,))

pl_id = 'spotify:playlist:37i9dQZEVXbMDoHDwVN2tF'
offset = 0

# load the first 100 songs
tracks = []
result = sp.playlist_items(pl_id, additional_types=['track'])
tracks.extend(result['items'])

# if playlist is larger than 100 songs, continue loading it until end
while result['next']:
    result = sp.next(result)
    tracks.extend(result['items'])

# remove all local songs
i = 0  # just for counting how many tracks are local
for item in tracks:
    if item['is_local']:
        tracks.remove(item)
        i += 1


# print result
print("Playlist length: " + str(len(tracks)) + "\nExcluding: " + str(i))

In [ ]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id,
                                               client_secret,
                                               redirect_uri,))
result = sp.search("uri") #search query
result['tracks']['items'][0]['artists']

### CSV and Database Portion of Code 2022

In [3]:
import json
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine
from sqlalchemy import inspect
import psycopg2
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials
import sys
import json

from config import client_id
from config import client_secret
from config import redirect_uri
from config import username
from config import password
from config import host
from config import db_password

import time
%load_ext sql

In [2]:
spotify_file = 'C:\\Users\\Blaine\\Documents\\College\\Bootcamp\\20) Final-Project\\Resources\\regional-global-weekly-2022-07-14.csv'

In [3]:
spotify_metadata = pd.read_csv(spotify_file, low_memory=False)

In [4]:
# Data is already clean
spotify_df = pd.DataFrame(spotify_metadata)
spotify_df.head()

,rank,uri,artist_names,track_name,source,peak_rank,previous_rank,weeks_on_chart,streams
0,1,spotify:track:75FEaRjZTKLhTrFGsfMUXR,Kate Bush,Running Up That Hill A Deal With God 2018 Rema...,Rhino,1,1,7,46609146
1,2,spotify:track:4Dvkj6JhhA12EX05fT7y2e,Harry Styles,As It Was,Columbia,1,2,15,44759969
2,3,spotify:track:2tTmW7RDtMQtBk7m2rYeSw,Bizarrap Quevedo,Quevedo Bzrp Music Sessions Vol 52,DALE PLAY Records,3,-1,1,38443942
3,4,spotify:track:6Sq7ltF9Qa7SNFBsV5Cogx,Bad Bunny Chencho Corleone,Me Porto Bonito,Rimas Entertainment LLC,2,4,10,38042392
4,5,spotify:track:6xGruZOHLs39ZbVccQTuPZ,Joji,Glimpse of Us,88rising Music Warner Records,1,3,5,34930156


In [5]:
# Setting the index
sp_df = spotify_df.set_index('rank')
sp_df

In [30]:
# Create the output file (CSV).
output_data_file = "C:\\Users\\Blaine\\Documents\\College\\Bootcamp\\20) Final-Project\\Resources\\cleaned_regional-global-weekly-2022-07-14.csv"
# Export the City_Data into a CSV.
sp_df.to_csv(output_data_file, index= True)

In [7]:
sp_genre_df = sp_df[["uri", "artist_names", "track_name", "source", "peak_rank", "previous_rank", "weeks_on_chart", "streams"]].copy()
sp_genre_df["genre"] = ""
sp_genre_df.head(10)

,uri,artist_names,track_name,source,peak_rank,previous_rank,weeks_on_chart,streams,genre
rank,,,,,,,,,
1,spotify:track:75FEaRjZTKLhTrFGsfMUXR,Kate Bush,Running Up That Hill A Deal With God 2018 Rema...,Rhino,1,1,7,46609146,
2,spotify:track:4Dvkj6JhhA12EX05fT7y2e,Harry Styles,As It Was,Columbia,1,2,15,44759969,
3,spotify:track:2tTmW7RDtMQtBk7m2rYeSw,Bizarrap Quevedo,Quevedo Bzrp Music Sessions Vol 52,DALE PLAY Records,3,-1,1,38443942,
4,spotify:track:6Sq7ltF9Qa7SNFBsV5Cogx,Bad Bunny Chencho Corleone,Me Porto Bonito,Rimas Entertainment LLC,2,4,10,38042392,
5,spotify:track:6xGruZOHLs39ZbVccQTuPZ,Joji,Glimpse of Us,88rising Music Warner Records,1,3,5,34930156,
6,spotify:track:1IHWl5LamUGEuP4ozKQSXZ,Bad Bunny,Titi Me Pregunto,Rimas Entertainment LLC,5,5,10,32423071,
7,spotify:track:3k3NWokhRRkEPhCzPmV8TW,Bad Bunny Bomba Estereo,Ojitos Lindos,Rimas Entertainment LLC,3,6,10,28065267,
8,spotify:track:5Eax0qFko2dh7Rl2lYs3bx,Bad Bunny,Efecto,Rimas Entertainment LLC,7,8,10,26558071,
9,spotify:track:6Xom58OOXk2SoU711L2IXO,Bad Bunny,Moscow Mule,Rimas Entertainment LLC,2,9,10,23671403,


In [8]:
# Creating empty list to hold both artist name and genre
artist_genre = []
print("Beginning Data Retrieval")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Data collection
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id,
                                               client_secret,
                                               redirect_uri,))
                                               
for index, row in sp_df.iterrows():
    artist = row["artist_names"]

    result = sp.search(artist)
    track = result['tracks']['items'][0]

    artist = sp.artist(track["artists"][0]["external_urls"]["spotify"])
    print("artist_name:",artist["name"] ,"artist genres:", artist["genres"])
    try: artist_genre.append({"artist_name": artist["name"],
                        "artist_genres": artist["genres"]
})
    except:
        print("Error finding artist and genere")
        pass

Beginning Data Retrieval
-----------------------------
artist_name: Kate Bush artist genres: ['art pop', 'art rock', 'baroque pop', 'new wave pop', 'permanent wave', 'piano rock', 'singer-songwriter']
artist_name: Harry Styles artist genres: ['pop']
artist_name: Bizarrap artist genres: ['argentine hip hop', 'pop venezolano', 'trap argentino']
artist_name: Bad Bunny artist genres: ['latin', 'reggaeton', 'trap latino']
artist_name: Joji artist genres: ['viral pop']
artist_name: Bad Bunny artist genres: ['latin', 'reggaeton', 'trap latino']
artist_name: Bad Bunny artist genres: ['latin', 'reggaeton', 'trap latino']
artist_name: Bad Bunny artist genres: ['latin', 'reggaeton', 'trap latino']
artist_name: Bad Bunny artist genres: ['latin', 'reggaeton', 'trap latino']
artist_name: Charlie Puth artist genres: ['dance pop', 'pop', 'viral pop']
artist_name: Glass Animals artist genres: ['gauze pop', 'indietronica', 'shiver pop']
artist_name: KAROL G artist genres: ['latin', 'reggaeton', 'reggaet

In [9]:
# Check for completion
len(artist_genre)

200

In [10]:
artist_genre_df = pd.DataFrame(artist_genre)
artist_genre_df.head(10)

,artist_name,artist_genres
0,Kate Bush,"[art pop, art rock, baroque pop, new wave pop,..."
1,Harry Styles,[pop]
2,Bizarrap,"[argentine hip hop, pop venezolano, trap argen..."
3,Bad Bunny,"[latin, reggaeton, trap latino]"
4,Joji,[viral pop]
5,Bad Bunny,"[latin, reggaeton, trap latino]"
6,Bad Bunny,"[latin, reggaeton, trap latino]"
7,Bad Bunny,"[latin, reggaeton, trap latino]"
8,Bad Bunny,"[latin, reggaeton, trap latino]"
9,Charlie Puth,"[dance pop, pop, viral pop]"


In [12]:
clean_artist_genre_df = artist_genre_df.set_index('artist_name')

In [13]:
clean_artist_genre_df.head(10)

,artist_genres
artist_name,
Kate Bush,"[art pop, art rock, baroque pop, new wave pop,..."
Harry Styles,[pop]
Bizarrap,"[argentine hip hop, pop venezolano, trap argen..."
Bad Bunny,"[latin, reggaeton, trap latino]"
Joji,[viral pop]
Bad Bunny,"[latin, reggaeton, trap latino]"
Bad Bunny,"[latin, reggaeton, trap latino]"
Bad Bunny,"[latin, reggaeton, trap latino]"
Bad Bunny,"[latin, reggaeton, trap latino]"


In [38]:
# Create the output file (CSV).
output_data_file = "C:\\Users\\Blaine\\Documents\\College\\Bootcamp\\20) Final-Project\\Resources\\artist_genre_2021.csv"
# Export the City_Data into a CSV.
artist_genre_df.to_csv(output_data_file, index= True)

In [44]:
artist_file = 'C:\\Users\\Blaine\\Documents\\College\\Bootcamp\\20) Final-Project\\Resources\\cleaned_artist_genre2022-07-14.csv'

In [45]:
artist_metadata = pd.read_csv(artist_file, low_memory=False)

In [46]:
artist_df = pd.DataFrame(artist_metadata)
artist_df.head()

,artist_name,artist_genres
0,Kate Bush,['pop']
1,Harry Styles,['pop']
2,Bizarrap,['hip hop']
3,Bad Bunny,['latin']
4,Joji,['pop']


In [47]:
db_string = f"postgresql+psycopg2://{username}:{password}@{host}/final_project"

In [48]:
engine = create_engine(db_string)

In [49]:
# Created connection based on engine
connection = engine

In [50]:
# This cell is working
artist_df.to_sql('artist_genres', con = engine, schema="public", if_exists="replace", index=False)

200

### Final Update to Genre table

In [1]:
import json
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine
from sqlalchemy import inspect
import psycopg2
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials
import sys
import json

from config import client_id
from config import client_secret
from config import redirect_uri
from config import username
from config import password
from config import host
from config import db_password

import time
%load_ext sql

In [3]:
artist_file = 'C:\\Users\\Blaine\\Documents\\College\\Bootcamp\\20) Final-Project\\Resources\\cleaned_artist_genres_full.csv'
artist_metadata = pd.read_csv(artist_file, low_memory=False)

In [4]:
artist_df = pd.DataFrame(artist_metadata)
artist_df.head()

,artist_name,artist_genres
0,2 Chainz,['hip hop']
1,24kGoldn,['pop']
2,5 Seconds of Summer,['pop']
3,6ix9ine,['rap']
4,A Boogie Wit da Hoodie,['rap']


In [5]:
db_string = f"postgresql+psycopg2://{username}:{password}@{host}/final_project"
engine = create_engine(db_string)
# Created connection based on engine
connection = engine

In [6]:
# This cell is working
artist_df.to_sql('artist_genres', con = engine, schema="public", if_exists="replace", index=False)

371

### Final Update to Song Table

In [1]:
import json
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine
from sqlalchemy import inspect
import psycopg2
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials
import sys
import json

from config import client_id
from config import client_secret
from config import redirect_uri
from config import username
from config import password
from config import host
from config import db_password

import time
%load_ext sql

In [2]:
song_file = 'C:\\Users\\Blaine\\Documents\\College\\Bootcamp\\20) Final-Project\\Resources\\regional_global_weekly_full.csv'
song_metadata = pd.read_csv(song_file, low_memory=False)

In [4]:
song_df = pd.DataFrame(song_metadata)
song_df.head()

,rank,uri,artist_names,track_name,source,peak_rank,previous_rank,weeks_on_chart,streams,week
0,1,spotify:track:75FEaRjZTKLhTrFGsfMUXR,Kate Bush,Running Up That Hill A Deal With God 2018 Rema...,Rhino,1,1,7,46609146,7/14/2022
1,2,spotify:track:4Dvkj6JhhA12EX05fT7y2e,Harry Styles,As It Was,Columbia,1,2,15,44759969,7/14/2022
2,3,spotify:track:2tTmW7RDtMQtBk7m2rYeSw,Bizarrap,Quevedo Bzrp Music Sessions Vol 52,DALE PLAY Records,3,-1,1,38443942,7/14/2022
3,4,spotify:track:6Sq7ltF9Qa7SNFBsV5Cogx,Bad Bunny,Me Porto Bonito,Rimas Entertainment LLC,2,4,10,38042392,7/14/2022
4,5,spotify:track:6xGruZOHLs39ZbVccQTuPZ,Joji,Glimpse of Us,88rising Music Warner Records,1,3,5,34930156,7/14/2022


In [5]:
db_string = f"postgresql+psycopg2://{username}:{password}@{host}/final_project"
engine = create_engine(db_string)
# Created connection based on engine
connection = engine

In [7]:
# This cell is working
song_df.to_sql('weekly_global_streams', con = engine, schema="public", if_exists="replace", index=False)

1000

### 2021

In [18]:
import json
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine
from sqlalchemy import inspect
import psycopg2
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials
import sys
import json

from config import client_id
from config import client_secret
from config import redirect_uri
from config import username
from config import password
from config import host
from config import db_password

import time
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [19]:
artist_file = 'C:\\Users\\Blaine\\Documents\\College\\Bootcamp\\20) Final-Project\\Resources\\artist_genre_2021.csv'

In [20]:
spotify_metadata = pd.read_csv(spotify_file, low_memory=False)

In [21]:
# Data is already clean
spotify_df = pd.DataFrame(spotify_metadata)
spotify_df.head()

,rank,uri,artist_names,track_name,source,peak_rank,previous_rank,weeks_on_chart,streams
0,1,spotify:track:3Wrjm47oTz2sjIgck11l5e,Maneskin,Beggin,RCA Records Label,1,1,6,52160401
1,2,spotify:track:4ZtFanR9U6ndgddUvNcjcG,Olivia Rodrigo,good 4 u,Olivia Rodrigo PS,1,2,9,46014325
2,3,spotify:track:5HCyWlXZPP0y6Gqq8TgA20,The Kid LAROI,STAY with Justin Bieber,Columbia,3,-1,1,43686697
3,4,spotify:track:6PQ88X9TkUIAUIZJHW2upE,Ed Sheeran,Bad Habits,Atlantic Records UK,3,3,3,38474834
4,5,spotify:track:0LThjFY2iTtNdd4wviwVV2,BTS,Permission to Dance,BIGHIT MUSIC,5,-1,1,38023318


In [22]:
# Setting the index
sp_df = spotify_df.set_index('rank')
sp_df

,uri,artist_names,track_name,source,peak_rank,previous_rank,weeks_on_chart,streams
rank,,,,,,,,
1,spotify:track:3Wrjm47oTz2sjIgck11l5e,Maneskin,Beggin,RCA Records Label,1,1,6,52160401
2,spotify:track:4ZtFanR9U6ndgddUvNcjcG,Olivia Rodrigo,good 4 u,Olivia Rodrigo PS,1,2,9,46014325
3,spotify:track:5HCyWlXZPP0y6Gqq8TgA20,The Kid LAROI,STAY with Justin Bieber,Columbia,3,-1,1,43686697
4,spotify:track:6PQ88X9TkUIAUIZJHW2upE,Ed Sheeran,Bad Habits,Atlantic Records UK,3,3,3,38474834
5,spotify:track:0LThjFY2iTtNdd4wviwVV2,BTS,Permission to Dance,BIGHIT MUSIC,5,-1,1,38023318
...,...,...,...,...,...,...,...,...
196,spotify:track:5RAIMjdrCEjpjaR5tBATXU,Maluma,Aloha,Hear This Music,95,174,13,5036030
197,spotify:track:5zqObw7wjBgL9TDiAymxPn,Anne-Marie,Our Song,Atlantic Records,100,187,8,4976121
198,spotify:track:0giiXonRhODYy4J1iy4Lb3,Rauw Alejandro,Sexo Virtual,Sony Music Latin,184,184,2,4975786


In [23]:
# Creating empty list to hold both artist name and genre
artist_genre = []
print("Beginning Data Retrieval")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Data collection
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id,
                                               client_secret,
                                               redirect_uri,))
                                               
for index, row in sp_df.iterrows():
    artist = row["artist_names"]

    result = sp.search(artist)
    track = result['tracks']['items'][0]

    artist = sp.artist(track["artists"][0]["external_urls"]["spotify"])
    print("artist_name:",artist["name"] ,"artist genres:", artist["genres"])
    try: artist_genre.append({"artist_name": artist["name"],
                        "artist_genres": artist["genres"]
})
    except:
        print("Error finding artist and genere")
        pass

Beginning Data Retrieval
-----------------------------
artist_name: Måneskin artist genres: ['indie rock italiano', 'italian pop']
artist_name: Olivia Rodrigo artist genres: ['pop']
artist_name: The Kid LAROI artist genres: ['australian hip hop']
artist_name: Ed Sheeran artist genres: ['pop', 'uk pop']
artist_name: Charlie Puth artist genres: ['dance pop', 'pop', 'viral pop']
artist_name: Lil Nas X artist genres: ['lgbtq+ hip hop', 'pop']
artist_name: Bad Bunny artist genres: ['latin', 'reggaeton', 'trap latino']
artist_name: Bad Bunny artist genres: ['latin', 'reggaeton', 'trap latino']
artist_name: Charlie Puth artist genres: ['dance pop', 'pop', 'viral pop']
artist_name: Post Malone artist genres: ['dfw rap', 'melodic rap', 'rap']
artist_name: Måneskin artist genres: ['indie rock italiano', 'italian pop']
artist_name: J Balvin artist genres: ['latin', 'reggaeton', 'reggaeton colombiano', 'trap latino']
artist_name: Jack Harlow artist genres: ['deep underground hip hop', 'kentucky hi

In [24]:
# Check for completion
len(artist_genre)

200

In [25]:
artist_genre_df = pd.DataFrame(artist_genre)
artist_genre_df.head(10)

,artist_name,artist_genres
0,Måneskin,"[indie rock italiano, italian pop]"
1,Olivia Rodrigo,[pop]
2,The Kid LAROI,[australian hip hop]
3,Ed Sheeran,"[pop, uk pop]"
4,Charlie Puth,"[dance pop, pop, viral pop]"
5,Lil Nas X,"[lgbtq+ hip hop, pop]"
6,Bad Bunny,"[latin, reggaeton, trap latino]"
7,Bad Bunny,"[latin, reggaeton, trap latino]"
8,Charlie Puth,"[dance pop, pop, viral pop]"
9,Post Malone,"[dfw rap, melodic rap, rap]"


In [26]:
artist_genre_df = artist_genre_df.set_index('artist_name')

In [27]:
# Create the output file (CSV).
output_data_file = "C:\\Users\\Blaine\\Documents\\College\\Bootcamp\\20) Final-Project\\Resources\\artist_genre_2021.csv"
# Export the City_Data into a CSV.
artist_genre_df.to_csv(output_data_file, index= True)

In [51]:
artist_file = 'C:\\Users\\Blaine\\Documents\\College\\Bootcamp\\20) Final-Project\\Resources\\artist_genre_2021.csv'

In [52]:
artist_metadata = pd.read_csv(artist_file, low_memory=False)

In [53]:
artist_df = pd.DataFrame(artist_metadata)
artist_df.head()

,artist_name,artist_genres
0,Maneskin,['pop']
1,Olivia Rodrigo,['pop']
2,The Kid LAROI,['hip hop']
3,Ed Sheeran,['pop']
4,BTS,['k pop']


In [54]:
db_string = f"postgresql+psycopg2://{username}:{password}@{host}/final_project"

In [55]:
engine = create_engine(db_string)

In [56]:
# Created connection based on engine
connection = engine

In [57]:
# This cell is working
artist_df.to_sql('artist_genres', con = engine, schema="public", if_exists="append", index=False)

200

### 2020

In [60]:
import json
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine
from sqlalchemy import inspect
import psycopg2
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials
import sys
import json

from config import client_id
from config import client_secret
from config import redirect_uri
from config import username
from config import password
from config import host
from config import db_password

import time
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [61]:
spotify_file = 'C:\\Users\\Blaine\\Documents\\College\\Bootcamp\\20) Final-Project\\Resources\\regional-global-weekly-2020-07-16.csv'
spotify_metadata = pd.read_csv(spotify_file, low_memory=False)

In [62]:
# Data is already clean
spotify_df = pd.DataFrame(spotify_metadata)
spotify_df.head()

,rank,uri,artist_names,track_name,source,peak_rank,previous_rank,weeks_on_chart,streams,week
0,1,spotify:track:7ytR5pFWmSjzHJIeQkgog4,DaBaby,ROCKSTAR feat Roddy Ricch,Interscope Records,1,1,13,36999793,7/16/2020
1,2,spotify:track:1xQ6trAsedVPCdbtDAmk0c,Jawsh 685,Savage Love Laxed Siren Beat,Columbia,2,2,5,34262951,7/16/2020
2,3,spotify:track:0VjIjW4GlUZAMYd2vXMi3b,The Weeknd,Blinding Lights,Republic Records,1,3,33,30727192,7/16/2020
3,4,spotify:track:7y7w4tl4MaRC2UMEj1mPtr,Juice WRLD,Come and Go with Marshmello,Interscope Records,4,-1,1,30027587,7/16/2020
4,5,spotify:track:6UelLqGlWMcVH1E5c4H7lY,Harry Styles,Watermelon Sugar,Columbia,5,5,35,26440765,7/16/2020


In [63]:
# Setting the index
sp_df = spotify_df.set_index('rank')
sp_df

,uri,artist_names,track_name,source,peak_rank,previous_rank,weeks_on_chart,streams,week
rank,,,,,,,,,
1,spotify:track:7ytR5pFWmSjzHJIeQkgog4,DaBaby,ROCKSTAR feat Roddy Ricch,Interscope Records,1,1,13,36999793,7/16/2020
2,spotify:track:1xQ6trAsedVPCdbtDAmk0c,Jawsh 685,Savage Love Laxed Siren Beat,Columbia,2,2,5,34262951,7/16/2020
3,spotify:track:0VjIjW4GlUZAMYd2vXMi3b,The Weeknd,Blinding Lights,Republic Records,1,3,33,30727192,7/16/2020
4,spotify:track:7y7w4tl4MaRC2UMEj1mPtr,Juice WRLD,Come and Go with Marshmello,Interscope Records,4,-1,1,30027587,7/16/2020
5,spotify:track:6UelLqGlWMcVH1E5c4H7lY,Harry Styles,Watermelon Sugar,Columbia,5,5,35,26440765,7/16/2020
...,...,...,...,...,...,...,...,...,...
196,spotify:track:5EbtodsuLbxrFDH6j5avVS,Camilo,El Mismo Aire con Pablo Alborán,Sony Music Latin,196,-1,1,4556759,7/16/2020
197,spotify:track:51Fjme0JiitpyXKuyQiCDo,Y2K,Lalala,Columbia,4,169,56,4541965,7/16/2020
198,spotify:track:7sQKy5vlPQllr0k9IjYJv3,Arcangel,Sigues Con El,Rimas Entertainment LLC,21,188,30,4538735,7/16/2020


In [64]:
# Creating empty list to hold both artist name and genre
artist_genre = []
print("Beginning Data Retrieval")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Data collection
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id,
                                               client_secret,
                                               redirect_uri,))
                                               
for index, row in sp_df.iterrows():
    artist = row["artist_names"]

    result = sp.search(artist)
    track = result['tracks']['items'][0]

    artist = sp.artist(track["artists"][0]["external_urls"]["spotify"])
    print("artist_name:",artist["name"] ,"artist genres:", artist["genres"])
    try: artist_genre.append({"artist_name": artist["name"],
                        "artist_genres": artist["genres"]
})
    except:
        print("Error finding artist and genere")
        pass

Beginning Data Retrieval
-----------------------------
artist_name: Dua Lipa artist genres: ['dance pop', 'pop', 'uk pop']
artist_name: Jawsh 685 artist genres: ['nz pop']
artist_name: The Weeknd artist genres: ['canadian contemporary r&b', 'canadian pop', 'pop']
artist_name: Juice WRLD artist genres: ['chicago rap', 'melodic rap']
artist_name: Harry Styles artist genres: ['pop']
artist_name: SAINt JHN artist genres: ['melodic rap', 'pop rap', 'rap', 'slap house', 'trap']
artist_name: Juice WRLD artist genres: ['chicago rap', 'melodic rap']
artist_name: Juice WRLD artist genres: ['chicago rap', 'melodic rap']
artist_name: Juice WRLD artist genres: ['chicago rap', 'melodic rap']
artist_name: StaySolidRocky artist genres: ['melodic rap']
artist_name: Duster artist genres: ['dream pop', 'slowcore']
artist_name: Powfu artist genres: ['sad rap']
artist_name: Lady Gaga artist genres: ['art pop', 'dance pop', 'pop']
artist_name: Juice WRLD artist genres: ['chicago rap', 'melodic rap']
artist_

In [65]:
# Check for completion
len(artist_genre)

200

In [66]:
artist_genre_df = pd.DataFrame(artist_genre)
artist_genre_df.head(10)

,artist_name,artist_genres
0,Dua Lipa,"[dance pop, pop, uk pop]"
1,Jawsh 685,[nz pop]
2,The Weeknd,"[canadian contemporary r&b, canadian pop, pop]"
3,Juice WRLD,"[chicago rap, melodic rap]"
4,Harry Styles,[pop]
5,SAINt JHN,"[melodic rap, pop rap, rap, slap house, trap]"
6,Juice WRLD,"[chicago rap, melodic rap]"
7,Juice WRLD,"[chicago rap, melodic rap]"
8,Juice WRLD,"[chicago rap, melodic rap]"
9,StaySolidRocky,[melodic rap]


In [67]:
artist_genre_df = artist_genre_df.set_index('artist_name')

In [68]:
# Create the output file (CSV).
output_data_file = "C:\\Users\\Blaine\\Documents\\College\\Bootcamp\\20) Final-Project\\Resources\\artist_genre_2020.csv"
# Export the City_Data into a CSV.
artist_genre_df.to_csv(output_data_file, index= True)

In [70]:
artist_file = 'C:\\Users\\Blaine\\Documents\\College\\Bootcamp\\20) Final-Project\\Resources\\artist_genre_2020.csv'
artist_metadata = pd.read_csv(artist_file, low_memory=False)

In [71]:
artist_df = pd.DataFrame(artist_metadata)
artist_df.head()

,artist_name,artist_genres
0,DaBaby,['pop']
1,Jawsh 685,['pop']
2,The Weeknd,['pop']
3,Juice WRLD,['rap']
4,Harry Styles,['pop']


In [72]:
db_string = f"postgresql+psycopg2://{username}:{password}@{host}/final_project"

In [73]:
engine = create_engine(db_string)

In [74]:
# Created connection based on engine
connection = engine

In [75]:
# This cell is working
artist_df.to_sql('artist_genres', con = engine, schema="public", if_exists="append", index=False)

200

### 2019

In [76]:
import json
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine
from sqlalchemy import inspect
import psycopg2
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials
import sys
import json

from config import client_id
from config import client_secret
from config import redirect_uri
from config import username
from config import password
from config import host
from config import db_password

import time
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [77]:
spotify_file = 'C:\\Users\\Blaine\\Documents\\College\\Bootcamp\\20) Final-Project\\Resources\\regional-global-weekly-2019-07-18.csv'
spotify_metadata = pd.read_csv(spotify_file, low_memory=False)

In [78]:
# Data is already clean
spotify_df = pd.DataFrame(spotify_metadata)
spotify_df.head()

,rank,uri,artist_names,track_name,source,peak_rank,previous_rank,weeks_on_chart,streams,week
0,1,spotify:track:0TK2YIli7K1leLovkQiNik,Shawn Mendes,Senorita,Island Records,1,1,4,64942021,7/18/2019
1,2,spotify:track:0hVXuCcriWRGvwMV1r5Yn9,Ed Sheeran,I Dont Care with Justin Bieber,Atlantic Records UK,1,3,10,38459324,7/18/2019
2,3,spotify:track:70eFcWOvlMObDhURTqT4Fv,Ed Sheeran,Beautiful People feat Khalid,Atlantic Records UK,3,5,3,36721702,7/18/2019
3,4,spotify:track:6vBdBCoOhKHiYDDOcorfNo,Post Malone,Goodbyes feat Young Thug,Republic Records,2,2,2,32776195,7/18/2019
4,5,spotify:track:2Fxmhks0bxGSBdJ92vM42m,Billie Eilish,bad guy,Interscope Records,1,4,16,27733366,7/18/2019


In [79]:
# Setting the index
sp_df = spotify_df.set_index('rank')
sp_df

,uri,artist_names,track_name,source,peak_rank,previous_rank,weeks_on_chart,streams,week
rank,,,,,,,,,
1,spotify:track:0TK2YIli7K1leLovkQiNik,Shawn Mendes,Senorita,Island Records,1,1,4,64942021,7/18/2019
2,spotify:track:0hVXuCcriWRGvwMV1r5Yn9,Ed Sheeran,I Dont Care with Justin Bieber,Atlantic Records UK,1,3,10,38459324,7/18/2019
3,spotify:track:70eFcWOvlMObDhURTqT4Fv,Ed Sheeran,Beautiful People feat Khalid,Atlantic Records UK,3,5,3,36721702,7/18/2019
4,spotify:track:6vBdBCoOhKHiYDDOcorfNo,Post Malone,Goodbyes feat Young Thug,Republic Records,2,2,2,32776195,7/18/2019
5,spotify:track:2Fxmhks0bxGSBdJ92vM42m,Billie Eilish,bad guy,Interscope Records,1,4,16,27733366,7/18/2019
...,...,...,...,...,...,...,...,...,...
196,spotify:track:5JEx7HbmvHQQswJCsoo9rA,Ellie Goulding,Close To Me with Diplo feat Swae Lee,Polydor Records,19,181,38,4500932,7/18/2019
197,spotify:track:1tC5WMSlsGDFfqnrquB1ty,Beyoncé,SPIRIT From Disneys The Lion King,Columbia,197,-1,1,4491520,7/18/2019
198,spotify:track:3QGsuHI8jO1Rx4JWLUh9jd,Shawn Mendes,Treat You Better,Island Records,26,195,53,4490711,7/18/2019


In [80]:
# Creating empty list to hold both artist name and genre
artist_genre = []
print("Beginning Data Retrieval")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Data collection
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id,
                                               client_secret,
                                               redirect_uri,))
                                               
for index, row in sp_df.iterrows():
    artist = row["artist_names"]

    result = sp.search(artist)
    track = result['tracks']['items'][0]

    artist = sp.artist(track["artists"][0]["external_urls"]["spotify"])
    print("artist_name:",artist["name"] ,"artist genres:", artist["genres"])
    try: artist_genre.append({"artist_name": artist["name"],
                        "artist_genres": artist["genres"]
})
    except:
        print("Error finding artist and genere")
        pass

Beginning Data Retrieval
-----------------------------
artist_name: Shawn Mendes artist genres: ['canadian pop', 'pop', 'viral pop']
artist_name: Ed Sheeran artist genres: ['pop', 'uk pop']
artist_name: Ed Sheeran artist genres: ['pop', 'uk pop']
artist_name: Post Malone artist genres: ['dfw rap', 'melodic rap', 'rap']
artist_name: Armani White artist genres: ['indie hip hop']
artist_name: Lil Nas X artist genres: ['lgbtq+ hip hop', 'pop']
artist_name: Ed Sheeran artist genres: ['pop', 'uk pop']
artist_name: Bad Bunny artist genres: ['latin', 'reggaeton', 'trap latino']
artist_name: Armani White artist genres: ['indie hip hop']
artist_name: Ed Sheeran artist genres: ['pop', 'uk pop']
artist_name: Johannes Brahms artist genres: ['classical', 'german romanticism', 'late romantic era']
artist_name: Drake artist genres: ['canadian hip hop', 'canadian pop', 'hip hop', 'rap', 'toronto rap']
artist_name: Lil Nas X artist genres: ['lgbtq+ hip hop', 'pop']
artist_name: Lewis Capaldi artist genr

In [81]:
# Check for completion
len(artist_genre)

200

In [82]:
artist_genre_df = pd.DataFrame(artist_genre)
artist_genre_df.head(10)

,artist_name,artist_genres
0,Shawn Mendes,"[canadian pop, pop, viral pop]"
1,Ed Sheeran,"[pop, uk pop]"
2,Ed Sheeran,"[pop, uk pop]"
3,Post Malone,"[dfw rap, melodic rap, rap]"
4,Armani White,[indie hip hop]
5,Lil Nas X,"[lgbtq+ hip hop, pop]"
6,Ed Sheeran,"[pop, uk pop]"
7,Bad Bunny,"[latin, reggaeton, trap latino]"
8,Armani White,[indie hip hop]
9,Ed Sheeran,"[pop, uk pop]"


In [83]:
artist_genre_df = artist_genre_df.set_index('artist_name')

In [84]:
# Create the output file (CSV).
output_data_file = "C:\\Users\\Blaine\\Documents\\College\\Bootcamp\\20) Final-Project\\Resources\\artist_genre_2019.csv"
# Export the City_Data into a CSV.
artist_genre_df.to_csv(output_data_file, index= True)

In [85]:
artist_file = 'C:\\Users\\Blaine\\Documents\\College\\Bootcamp\\20) Final-Project\\Resources\\artist_genre_2019.csv'
artist_metadata = pd.read_csv(artist_file, low_memory=False)

In [86]:
artist_df = pd.DataFrame(artist_metadata)
artist_df.head()

,artist_name,artist_genres
0,Shawn Mendes,['pop']
1,Ed Sheeran,['pop']
2,Ed Sheeran,['pop']
3,Post Malone,['rap']
4,Billie Eilish,['hip hop']


In [87]:
db_string = f"postgresql+psycopg2://{username}:{password}@{host}/final_project"
engine = create_engine(db_string)
# Created connection based on engine
connection = engine

In [88]:
# This cell is working
artist_df.to_sql('artist_genres', con = engine, schema="public", if_exists="append", index=False)

200

### 2018

In [89]:
import json
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine
from sqlalchemy import inspect
import psycopg2
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials
import sys
import json

from config import client_id
from config import client_secret
from config import redirect_uri
from config import username
from config import password
from config import host
from config import db_password

import time
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [90]:
spotify_file = 'C:\\Users\\Blaine\\Documents\\College\\Bootcamp\\20) Final-Project\\Resources\\regional-global-weekly-2018-07-19.csv'
spotify_metadata = pd.read_csv(spotify_file, low_memory=False)

In [91]:
# Data is already clean
spotify_df = pd.DataFrame(spotify_metadata)
spotify_df.head()

,rank,uri,artist_names,track_name,source,peak_rank,previous_rank,weeks_on_chart,streams,week
0,1,spotify:track:0bAkKNCQfWkexHFn7fIKns,Drake,In My Feelings,Cash Money,1,1,3,67499798,7/19/2018
1,2,spotify:track:7gkWXbAxIYuvtOpcN3p9GJ,Ariana Grande,God is a woman,Republic Records,2,-1,1,29847494,7/19/2018
2,3,spotify:track:7fa9MBXhVfQ8P8Df9OEbD8,Maroon 5,Girls Like You feat Cardi B Cardi B Version,Interscope Records,3,4,7,29296811,7/19/2018
3,4,spotify:track:36ONiya0OANYknz0GgJmwB,Drake,Don’t Matter To Me with Michael Jackson,Cash Money,2,2,3,28813636,7/19/2018
4,5,spotify:track:58q2HKrzhC3ozto2nDdN4z,Cardi B,I Like It,Atlantic Records,3,7,15,27369377,7/19/2018


In [92]:
# Setting the index
sp_df = spotify_df.set_index('rank')
sp_df

,uri,artist_names,track_name,source,peak_rank,previous_rank,weeks_on_chart,streams,week
rank,,,,,,,,,
1,spotify:track:0bAkKNCQfWkexHFn7fIKns,Drake,In My Feelings,Cash Money,1,1,3,67499798,7/19/2018
2,spotify:track:7gkWXbAxIYuvtOpcN3p9GJ,Ariana Grande,God is a woman,Republic Records,2,-1,1,29847494,7/19/2018
3,spotify:track:7fa9MBXhVfQ8P8Df9OEbD8,Maroon 5,Girls Like You feat Cardi B Cardi B Version,Interscope Records,3,4,7,29296811,7/19/2018
4,spotify:track:36ONiya0OANYknz0GgJmwB,Drake,Don’t Matter To Me with Michael Jackson,Cash Money,2,2,3,28813636,7/19/2018
5,spotify:track:58q2HKrzhC3ozto2nDdN4z,Cardi B,I Like It,Atlantic Records,3,7,15,27369377,7/19/2018
...,...,...,...,...,...,...,...,...,...
196,spotify:track:3jipFRgLyKK0oJoG1pKicx,Drake,Jaded,Cash Money,35,103,3,3947144,7/19/2018
197,spotify:track:2jyjhRf6DVbMPU5zxagN2h,Passenger,Let Her Go,Nettwerk Music Group,170,-1,10,3944815,7/19/2018
198,spotify:track:7oK9VyNzrYvRFo7nQEYkWN,The Killers,Mr Brightside,Island Records,148,-1,25,3934737,7/19/2018


In [93]:
# Creating empty list to hold both artist name and genre
artist_genre = []
print("Beginning Data Retrieval")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Data collection
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id,
                                               client_secret,
                                               redirect_uri,))
                                               
for index, row in sp_df.iterrows():
    artist = row["artist_names"]

    result = sp.search(artist)
    track = result['tracks']['items'][0]

    artist = sp.artist(track["artists"][0]["external_urls"]["spotify"])
    print("artist_name:",artist["name"] ,"artist genres:", artist["genres"])
    try: artist_genre.append({"artist_name": artist["name"],
                        "artist_genres": artist["genres"]
})
    except:
        print("Error finding artist and genere")
        pass

Beginning Data Retrieval
-----------------------------
artist_name: Drake artist genres: ['canadian hip hop', 'canadian pop', 'hip hop', 'rap', 'toronto rap']
artist_name: Ariana Grande artist genres: ['dance pop', 'pop']
artist_name: Maroon 5 artist genres: ['pop']
artist_name: Drake artist genres: ['canadian hip hop', 'canadian pop', 'hip hop', 'rap', 'toronto rap']
artist_name: Cardi B artist genres: ['dance pop', 'pop', 'pop rap', 'rap']
artist_name: Juice WRLD artist genres: ['chicago rap', 'melodic rap']
artist_name: Kanye West artist genres: ['chicago rap', 'rap']
artist_name: Tyga artist genres: ['dance pop', 'hip hop', 'pop', 'pop rap', 'rap', 'southern hip hop', 'trap']
artist_name: Calvin Harris artist genres: ['dance pop', 'edm', 'electro house', 'house', 'pop', 'progressive house', 'uk dance']
artist_name: Clean Bandit artist genres: ['dance pop', 'edm', 'pop', 'pop dance', 'tropical house', 'uk dance', 'uk funky', 'uk pop']
artist_name: Post Malone artist genres: ['dfw ra

In [94]:
# Check for completion
len(artist_genre)

200

In [95]:
artist_genre_df = pd.DataFrame(artist_genre)
artist_genre_df.head(10)

,artist_name,artist_genres
0,Drake,"[canadian hip hop, canadian pop, hip hop, rap,..."
1,Ariana Grande,"[dance pop, pop]"
2,Maroon 5,[pop]
3,Drake,"[canadian hip hop, canadian pop, hip hop, rap,..."
4,Cardi B,"[dance pop, pop, pop rap, rap]"
5,Juice WRLD,"[chicago rap, melodic rap]"
6,Kanye West,"[chicago rap, rap]"
7,Tyga,"[dance pop, hip hop, pop, pop rap, rap, southe..."
8,Calvin Harris,"[dance pop, edm, electro house, house, pop, pr..."
9,Clean Bandit,"[dance pop, edm, pop, pop dance, tropical hous..."


In [96]:
artist_genre_df = pd.DataFrame(artist_genre)
artist_genre_df.head(10)

,artist_name,artist_genres
0,Drake,"[canadian hip hop, canadian pop, hip hop, rap,..."
1,Ariana Grande,"[dance pop, pop]"
2,Maroon 5,[pop]
3,Drake,"[canadian hip hop, canadian pop, hip hop, rap,..."
4,Cardi B,"[dance pop, pop, pop rap, rap]"
5,Juice WRLD,"[chicago rap, melodic rap]"
6,Kanye West,"[chicago rap, rap]"
7,Tyga,"[dance pop, hip hop, pop, pop rap, rap, southe..."
8,Calvin Harris,"[dance pop, edm, electro house, house, pop, pr..."
9,Clean Bandit,"[dance pop, edm, pop, pop dance, tropical hous..."


In [97]:
artist_genre_df = artist_genre_df.set_index('artist_name')

In [98]:
# Create the output file (CSV).
output_data_file = "C:\\Users\\Blaine\\Documents\\College\\Bootcamp\\20) Final-Project\\Resources\\artist_genre_2018.csv"
# Export the City_Data into a CSV.
artist_genre_df.to_csv(output_data_file, index= True)

In [99]:
artist_file = 'C:\\Users\\Blaine\\Documents\\College\\Bootcamp\\20) Final-Project\\Resources\\artist_genre_2018.csv'
artist_metadata = pd.read_csv(artist_file, low_memory=False)

In [100]:
artist_df = pd.DataFrame(artist_metadata)
artist_df.head()

,artist_name,artist_genres
0,Drake,['hip hop']
1,Ariana Grande,['pop']
2,Maroon 5,['pop']
3,Drake,['hip hop']
4,Cardi B,['rap']


In [101]:
db_string = f"postgresql+psycopg2://{username}:{password}@{host}/final_project"
engine = create_engine(db_string)
# Created connection based on engine
connection = engine

In [102]:
# This cell is working
artist_df.to_sql('artist_genres', con = engine, schema="public", if_exists="append", index=False)

200

### Test code lines

In [59]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id,
                                               client_secret,
                                               redirect_uri,))
                                               
result = sp.search("Dan and Shay")
track = result['tracks']['items'][0]

artist = sp.artist(track["artists"][0]["external_urls"]["spotify"])
print("artist_name:",artist["name"] ,"artist genres:", artist["genres"])

album = sp.album(track["album"]["external_urls"]["spotify"])
print("album genres:", album["genres"])
print("album release-date:", album["release_date"])


# len(sp_df.artist_names)

artist_name: Dan + Shay artist genres: ['contemporary country', 'country', 'pop']
album genres: []
album release-date: 2021-08-13


In [32]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id,
                                               client_secret,
                                               redirect_uri,))
                                               
result = sp.search("Ramstein")
track = result['tracks']['items'][0]

artist = sp.artist(track["artists"][0]["external_urls"]["spotify"])
print("artist_name:",artist["name"] ,"artist genres:", artist["genres"])

album = sp.album(track["album"]["external_urls"]["spotify"])
print("album genres:", album["genres"])
print("album release-date:", album["release_date"])

artist_name: Rammstein artist genres: ['alternative metal', 'german metal', 'industrial', 'industrial metal', 'industrial rock', 'neue deutsche harte']
album genres: []
album release-date: 1997-08-25


In [ ]:
# Work to correct this code block later for quick appending of data
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id,
                                               client_secret,
                                               redirect_uri,))
                                               
for index, row in sp_df.iterrows():
    artist = row["artist_names"]

    result = sp.search(artist)
    track = result['tracks']['items'][0]

    artist = sp.artist(track["artists"][0]["external_urls"]["spotify"])
    print("artist genres:", artist["genres"])

# The below still needs work to add the genres to the dataframe.  Above does iterate through the list.

try:
    sp_genre_df.loc[index, "genre"] = artist["results"][0]["genre"]
except (IndexError):
    print("Artist and Genre not found... Skipping.")

In [ ]:
sp_genre_df = pd.DataFrame(columns=['genre'])
for i in range(200):
    sp_genre_df = sp_genre_df.append({'genre':i}, ignore_index=False)

sp_genre_df.head()

In [8]:
len(sp_df.columns)

8

In [12]:
len(sp_df.uri)

200